In [2]:
import requests
import pandas as pd

In [3]:
n_case_investigations = requests.get(
    """
    https://phl.carto.com/api/v2/sql?q=SELECT count(*) FROM case_investigations ci 
    """
).json()["rows"][0]["count"]
print(f"There are {n_case_investigations} investigations of violations in the dataset.")

n_unique_case_investigations = requests.get(
    """
    https://phl.carto.com/api/v2/sql?q=SELECT count(distinct(investigationprocessid)) FROM case_investigations ci 
    """
).json()["rows"][0]["count"]
print(
    f"There are {n_case_investigations} unique investigations of violations in the dataset."
)

There are 1518697 investigations of violations in the dataset.
There are 1518697 unique investigations of violations in the dataset.


In [4]:
case_investigations = requests.get(
    """
    https://phl.carto.com/api/v2/sql?q=SELECT * FROM case_investigations ci limit 1
    """
).json()["rows"]
case_investigations[0]

{'cartodb_id': 1,
 'the_geom': '0101000020E61000002D69BDF30DCA52C044EDAC8D23014440',
 'the_geom_webmercator': '0101000020110F0000EFDDAC915CEA5FC14FF8C9D428915241',
 'objectid': 66,
 'addressobjectid': '15330210',
 'parcel_id_num': '521288',
 'casenumber': '569113',
 'casetype': 'NOTICE OF VIOLATION',
 'caseresponsibility': 'CODE ENFORCEMENT INVESTIGATOR',
 'casepriority': 'STANDARD',
 'investigationprocessid': 212366007,
 'investigationtype': None,
 'investigationcompleted': '2020-08-04T13:40:58Z',
 'investigationstatus': 'PASSED',
 'opa_account_num': '131237500',
 'address': '3622 N BOUVIER ST',
 'unit_type': None,
 'unit_num': None,
 'zip': '19140-4024',
 'censustract': '201.02',
 'opa_owner': 'TYMELESS INVESTMENT LLC',
 'systemofrecord': 'ECLIPSE',
 'geocode_x': 2694727.6310132,
 'geocode_y': 256760.8127715,
 'council_district': None,
 'posse_jobid': '195194709'}

In [5]:
n_case_investigations_with_violation = requests.get(
    """
    https://phl.carto.com/api/v2/sql?q=SELECT count(distinct(investigationprocessid)) FROM case_investigations ci, violations v
    where ci.casenumber = v.casenumber
    """
).json()["rows"][0]["count"]
pct_with_violation = round(
    n_case_investigations_with_violation / n_case_investigations * 100, 2
)
print(
    f"There are {n_case_investigations_with_violation} ({pct_with_violation}%) investigations of violations that can be attached to violations in the dataset."
)

There are 1323834 (87.17%) investigations of violations that can be attached to violations in the dataset.


In [6]:
case_investigations_with_violation_results = requests.get(
    """
    https://phl.carto.com/api/v2/sql?q=SELECT ci.*, v.opa_account_num, v.violationcodetitle
    FROM case_investigations ci, violations v
    where ci.casenumber = v.casenumber  limit 100
    """
).json()["rows"]
# single case number can have multiple violations (casenumber=706411)
# single case number can have multiple investigations (205528)
pd.DataFrame(case_investigations_with_violation_results)

,cartodb_id,the_geom,the_geom_webmercator,objectid,addressobjectid,parcel_id_num,casenumber,casetype,caseresponsibility,casepriority,...,unit_num,zip,censustract,opa_owner,systemofrecord,geocode_x,geocode_y,council_district,posse_jobid,violationcodetitle
0,79,0101000020E610000032F6BFDECDCC52C0629AEDEE82FC...,0101000020110F0000930A9F3E08EF5FC1DCD3C2A3078C...,62,15897333,475137,568999,NOTICE OF VIOLATION,CSU INVESTIGATOR,UNSAFE,...,None,19104-1123,110,COLEMAN GREGORY,ECLIPSE,2.683078e+06,243246.473027,None,195194705,EXTERIOR STRUCT UNSAFE COND 8
1,1,0101000020E61000002D69BDF30DCA52C044EDAC8D2301...,0101000020110F0000EFDDAC915CEA5FC14FF8C9D42891...,66,15330210,521288,569113,NOTICE OF VIOLATION,CODE ENFORCEMENT INVESTIGATOR,STANDARD,...,None,19140-4024,201.02,TYMELESS INVESTMENT LLC,ECLIPSE,2.694728e+06,256760.812772,None,195194709,VACANT STRUCTURE LICENSE
2,2,0101000020E6100000983A5A5C69CD52C02B5C839662F8...,0101000020110F00006B5F715C10F05FC1143B153B7587...,67,15388160,448114,569236,NOTICE OF VIOLATION,CSU INVESTIGATOR,UNSAFE,...,None,19143-4405,74,MARVA L CLARK,ECLIPSE,2.680757e+06,231431.283136,None,195194712,UNSAFE STRUCTURE
3,2,0101000020E6100000983A5A5C69CD52C02B5C839662F8...,0101000020110F00006B5F715C10F05FC1143B153B7587...,67,15388160,448114,569236,NOTICE OF VIOLATION,CSU INVESTIGATOR,UNSAFE,...,None,19143-4405,74,MARVA L CLARK,ECLIPSE,2.680757e+06,231431.283136,None,195194712,EXTERIOR STRUCT UNSAFE COND 7
4,3,0101000020E6100000983A5A5C69CD52C02B5C839662F8...,0101000020110F00006B5F715C10F05FC1143B153B7587...,68,15388160,448114,569236,NOTICE OF VIOLATION,CSU INVESTIGATOR,UNSAFE,...,None,19143-4405,74,MARVA L CLARK,ECLIPSE,2.680757e+06,231431.283136,None,195194712,UNSAFE STRUCTURE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,173,0101000020E61000006C39E508EFCE52C058DDD36224FD...,0101000020110F0000012CF842A6F25FC112F71993BA8C...,170,133141757,405404,572213,NOTICE OF VIOLATION,CSU INVESTIGATOR,UNSAFE,...,None,19131-3544,113,BAKER FRED,ECLIPSE,2.673705e+06,244773.131909,None,195194942,EXTERIOR STRUCT UNSAFE COND 8
96,173,0101000020E61000006C39E508EFCE52C058DDD36224FD...,0101000020110F0000012CF842A6F25FC112F71993BA8C...,170,133141757,405404,572213,NOTICE OF VIOLATION,CSU INVESTIGATOR,UNSAFE,...,None,19131-3544,113,BAKER FRED,ECLIPSE,2.673705e+06,244773.131909,None,195194942,UNSAFE STRUCTURE
97,174,0101000020E61000006C39E508EFCE52C058DDD36224FD...,0101000020110F0000012CF842A6F25FC112F71993BA8C...,171,133141757,405404,572213,NOTICE OF VIOLATION,CSU INVESTIGATOR,UNSAFE,...,None,19131-3544,113,BAKER FRED,ECLIPSE,2.673705e+06,244773.131909,None,195194942,EXTERIOR STRUCT UNSAFE COND 8
98,174,0101000020E61000006C39E508EFCE52C058DDD36224FD...,0101000020110F0000012CF842A6F25FC112F71993BA8C...,171,133141757,405404,572213,NOTICE OF VIOLATION,CSU INVESTIGATOR,UNSAFE,...,None,19131-3544,113,BAKER FRED,ECLIPSE,2.673705e+06,244773.131909,None,195194942,UNSAFE STRUCTURE
